# Two jug problem with capacities x, y
## Formulation
>**State space**  
$$ S = \{(u, v) | u \in \{0,1,..,x\} \land v \in \{0, 1, ..., y\}\} $$

>**Actions**  
$ a1 : \text{empty jug-1} $
$ a2 : \text{empty jug-2} $
$$ a3 : \text{fill jug-1} $$
$$ a4 : \text{fill jug-2} $$
$$ a5 : \text{transfer from jug-1 to jug-2} $$
$$ a6 : \text{transfer from jug-2 to jug-1} $$ 

>**Successor function**  
$$ f : S \longrightarrow S $$
$$ f(s|a1) = (0, v) $$
$$ f(s|a2) =  (u, 0) $$
$$ f(s|a3) = (x, v) $$
$$ f(s|a4) = (u, y) $$
$$
f(s|a5)=
\begin{cases}
(0, v+u) & \quad \text{when $v+u \leq y$}\\ 
(u - (y-v), y) & \quad \text{otherwise}
\end{cases}
$$
$$
f(s|a6)=
\begin{cases}
(u+v, 0) & \quad \text{when $u+v \leq x$}\\ 
(x, v-(x-u)) & \quad \text{otherwise}
\end{cases}
$$


## Solution
> Exhaustive state space search with Dijkstra's single source shortest path algorithm.\
> Time complexity: $O(uva)$\
> Space complexity: $O(uv)$

In [828]:
x, y = 6, 4
cap_goal = 2
actions = [f"e{x}l", f"e{y}l", f"f{x}l", f"f{y}l", f"t{x}l{y}l", f"t{y}l{x}l"]
start = {x:0, y:0}

visited = [[False] * (y+1) for _ in range(x+1)] # visited[3+1][4+1]
visited[0][0] = True

cost_array = [[float("inf")] * (y+1) for _ in range(x+1)] # cost_array[3+1][4+1]
cost_array[0][0] = 0

solution = {0:[start]}

In [829]:
def successor_function(state: dict, action):
    next = state.copy()
    if action == f"e{x}l":
        # empty x jug
        next[x] = 0
    elif action == f"e{y}l":
        # empty y jug
        next[y] = 0
    elif action == "f{x}l":
        # fill x jug
        next[x] = x
    elif action == f"f{y}l":
        # fill y jug
        next[y] = y
    elif action == f"t{x}l{y}l":
        # transfer from x jug to y jug
        space_left = y-state[y]
        if state[x] < space_left:
            next[y] += state[x]
            next[x] = 0
        else:
            next[x] -= space_left
            next[y] = y
    elif action == f"t{y}l{x}l":
        # transfer from y jug to x jug
        space_left = x-state[x]
        if state[y] < space_left:
            next[x] += state[y]
            next[y] = 0
        else:
            next[y] -= space_left
            next[x] = x    
    return next

In [830]:
def get_cost(state):
    return cost_array[state[x]][state[y]]
def set_cost(state, cost):
    cost_array[state[x]][state[y]] = cost

In [831]:
def comparator(state):
    return cost_array[state[x]][state[y]]

In [832]:
# greedy BFS a.k.a Dijkstra's
queue = [start]
visited[start[x]][start[y]] = True
while queue:
    state = queue.pop(0)

    for action in actions:
        next_state = successor_function(state, action)

        if not visited[next_state[x]][next_state[y]]:
            queue.append(next_state)
            visited[next_state[x]][next_state[y]] = True

        next_cost = get_cost(state) + 1
        if next_cost < get_cost(next_state):
            set_cost(next_state, next_cost)
            solution[next_cost] = solution.get(next_cost, [])
            solution[next_cost].append(next_state)
        
    # fix min priority queue
    queue = sorted(queue, key=comparator)


In [833]:
# solution extraction
def first_goal():
    for cost in solution:
        for state in solution[cost]:
            if cap_goal in state.values():
                return cost, state

def is_neighbor(state1, state2):
    for action in actions:
        if successor_function(state1, action) == state2:
            return True
    return False

def first_solution():
    cost, state = first_goal()
    res = [state]
    cost -= 1

    while cost in solution:
        for next_state in solution[cost]:
            if is_neighbor(next_state, state):
                res.insert(0, next_state)
                state = next_state
                cost -= 1
    return res

for step in first_solution()[:-1]:
    print(step, end=" -> ")
print(first_solution()[-1])
print(solution)

{6: 0, 4: 0} -> {6: 0, 4: 4} -> {6: 4, 4: 0} -> {6: 4, 4: 4} -> {6: 6, 4: 2}
{0: [{6: 0, 4: 0}], 1: [{6: 0, 4: 4}], 2: [{6: 4, 4: 0}], 3: [{6: 4, 4: 4}], 4: [{6: 6, 4: 2}], 5: [{6: 0, 4: 2}, {6: 6, 4: 0}, {6: 6, 4: 4}], 6: [{6: 2, 4: 0}, {6: 2, 4: 4}]}
